# Train a neural network with very little data

In this notebook, we will use Nobrainer to train a model with limited data. We will start off with a pre-trained model. You can find available pre-trained Nobrainer models at https://github.com/neuronets/nobrainer-models.

The pre-trained models can be used to train models for the same task as they were trained for or to transfer learn a new task. For instance, a pre-trained brain extraction model can be re-trained for tumor labeling. In this notebook, we will train a brain extraction model, but keep in mind that you can retrain these models for many 3D semantic segmentation tasks.

In the following cells, we will:

1. Get sample T1-weighted MR scans as features and FreeSurfer segmentations as labels.
    - We will binarize the FreeSurfer to get a precise brainmask.
2. Convert the data to TFRecords format.
3. Create two Datasets of the features and labels.
    - One dataset will be for training and the other will be for evaluation.
4. Load a pre-trained 3D semantic segmentation model.
5. Choose a loss function and metrics to use.
6. Train on part of the data.
7. Evaluate on the rest of the data.

## Google Colaboratory

If you are using Colab, please switch your runtime to GPU. To do this, select `Runtime > Change runtime type` in the top menu. Then select GPU under `Hardware accelerator`. A GPU greatly speeds up training.

In this tutorial we will need to pull in external models using datalad. To support it we first install git-annex using Neurodebian.

**Note: Sometimes getting the gpg key can fail. If you notice such an error, simply re-execute the cell.**

In [ ]:
!wget -O- http://neuro.debian.net/lists/bionic.us-nh.full | tee /etc/apt/sources.list.d/neurodebian.sources.list \
 && apt-key adv --recv-keys --keyserver hkp://pool.sks-keyservers.net:80 0xA5D32F012649A5A9 \
 && apt-get update \
 && apt-get install git-annex-standalone git

Let's make sure the correct version of git-annex is installed. This should be at least **8.20210223** or later.

In [ ]:
!git-annex version

In [ ]:
!pip install --no-cache-dir nobrainer nilearn datalad datalad-osf

In [ ]:
import nobrainer

# Get sample features and labels

We use 9 pairs of volumes for training and 1 pair of volumes for evaluation. Many more volumes would be required to train a model for any useful purpose.

In [ ]:
csv_of_filepaths = nobrainer.utils.get_data()
filepaths = nobrainer.io.read_csv(csv_of_filepaths)

train_paths = filepaths[:9]
evaluate_paths = filepaths[9:]

# Convert medical images to TFRecords

Remember how many full volumes are in the TFRecords files. This will be necessary to know how many steps are in on training epoch. The default training method needs to know this number, because Datasets don't always know how many items they contain.

In [ ]:
# Verify that all volumes have the same shape and that labels are integer-ish.

invalid = nobrainer.io.verify_features_labels(train_paths, num_parallel_calls=2)
assert not invalid

invalid = nobrainer.io.verify_features_labels(evaluate_paths)
assert not invalid

In [ ]:
!mkdir -p data

In [ ]:
# Convert training and evaluation data to TFRecords.

nobrainer.tfrecord.write(
    features_labels=train_paths,
    filename_template='data/data-train_shard-{shard:03d}.tfrec',
    examples_per_shard=3)

nobrainer.tfrecord.write(
    features_labels=evaluate_paths,
    filename_template='data/data-evaluate_shard-{shard:03d}.tfrec',
    examples_per_shard=1)

In [ ]:
!ls data

# Create Datasets

In [ ]:
n_classes = 1
batch_size = 2
volume_shape = (256, 256, 256)
block_shape = (128, 128, 128)
n_epochs = None
augment = False
shuffle_buffer_size = 10
num_parallel_calls = 2

In [ ]:
dataset_train = nobrainer.dataset.get_dataset(
    file_pattern='data/data-train_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=n_epochs,
    augment=augment,
    shuffle_buffer_size=shuffle_buffer_size,
    num_parallel_calls=num_parallel_calls,
)

dataset_evaluate = nobrainer.dataset.get_dataset(
    file_pattern='data/data-evaluate_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=1,
    augment=False,
    shuffle_buffer_size=None,
    num_parallel_calls=1,
)

In [ ]:
dataset_train

In [ ]:
dataset_evaluate

# Load pre-trained model

Use datalad to retrieve trained models.

In [ ]:
!datalad clone https://github.com/neuronets/trained-models && \
  cd trained-models && git-annex enableremote osf-storage && \
  datalad get -s osf-storage neuronets/brainy/0.1.0/brain-extraction-unet-128iso-model.h5

In [ ]:
import tensorflow as tf

In [ ]:
model_path = "trained-models/neuronets/brainy/0.1.0/brain-extraction-unet-128iso-model.h5"

In [ ]:
model = tf.keras.models.load_model(model_path, compile=False)

In [ ]:
model.summary()

# Considerations for transfer learning

Training a neural network changes the model's weights. A pre-trained network has learned weights for a task, and we do not want to forget these weights during training. In other words, we do not want to ruin the pre-trained weights when using our new data. To avoid dramatic changes in the learnable parameters, we can apply regularization and use a relatively small learning rate.

In [ ]:
for layer in model.layers:
    layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-05)

model.compile(
    optimizer=optimizer,
    loss=nobrainer.losses.jaccard,
    metrics=[nobrainer.metrics.dice],
)

# Train and evaluate model

$$
steps = \frac{nBlocks}{volume} * \frac{nVolumes}{batchSize}
$$

In [ ]:
steps_per_epoch = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(train_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

steps_per_epoch

In [ ]:
validation_steps = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(evaluate_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

validation_steps

In [ ]:
model.fit(
    dataset_train,
    epochs=5,
    steps_per_epoch=steps_per_epoch, 
    validation_data=dataset_evaluate, 
    validation_steps=validation_steps)

# Predict natively medical images (without TFRecords)

In [ ]:
from nobrainer.volume import standardize_numpy
import nibabel as nib

image_path = evaluate_paths[0][0]
out = nobrainer.prediction.predict_from_filepath(image_path, 
                                           model,
                                           block_shape = block_shape,
                                           batch_size = batch_size,
                                           normalizer = standardize_numpy,
                                             )
out.shape

In [ ]:
from nilearn import plotting
plotting.plot_roi(out, bg_img=image_path, alpha=0.4)